{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# CORD-19 Dataset Exploration\n",
    "## COVID-19 Research Papers Analysis\n",
    "\n",
    "This notebook explores the CORD-19 dataset to understand patterns in COVID-19 research publications."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Import required libraries\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "import numpy as np\n",
    "from wordcloud import WordCloud\n",
    "from collections import Counter\n",
    "import re\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')\n",
    "\n",
    "# Set visualization style\n",
    "sns.set_style('whitegrid')\n",
    "plt.rcParams['figure.figsize'] = (12, 6)\n",
    "\n",
    "print('Libraries imported successfully!')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Data Loading"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load the dataset\n",
    "# Use nrows parameter to load a sample for faster exploration\n",
    "df = pd.read_csv('metadata.csv', nrows=10000)\n",
    "\n",
    "print(f'Dataset loaded with {len(df)} rows and {len(df.columns)} columns')\n",
    "df.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Initial Data Exploration"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Display dataset information\n",
    "print('=== Dataset Info ===')\n",
    "df.info()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Check column names\n",
    "print('Column names:')\n",
    "print(df.columns.tolist())"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Display basic statistics\n",
    "df.describe()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Check for missing values\n",
    "missing_data = pd.DataFrame({\n",
    "    'Column': df.columns,\n",
    "    'Missing Count': df.isnull().sum().values,\n",
    "    'Missing Percentage': (df.isnull().sum().values / len(df) * 100)\n",
    "})\n",
    "\n",
    "missing_data = missing_data[missing_data['Missing Count'] > 0].sort_values(\n",
    "    'Missing Percentage', ascending=False\n",
    ")\n",
    "\n",
    "print('\\nMissing Data Analysis:')\n",
    "print(missing_data)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Visualize missing data\n",
    "plt.figure(figsize=(12, 6))\n",
    "missing_data.plot(x='Column', y='Missing Percentage', kind='bar', color='coral')\n",
    "plt.title('Missing Data by Column', fontsize=14, fontweight='bold')\n",
    "plt.xlabel('Column')\n",
    "plt.ylabel('Missing Percentage (%)')\n",
    "plt.xticks(rotation=45, ha='right')\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Data Cleaning"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create a clean copy\n",
    "df_clean = df.copy()\n",
    "\n",
    "# Convert publish_time to datetime\n",
    "df_clean['publish_time'] = pd.to_datetime(df_clean['publish_time'], errors='coerce')\n",
    "\n",
    "# Extract year\n",
    "df_clean['year'] = df_clean['publish_time'].dt.year\n",
    "\n",
    "# Remove rows without title\n",
    "df_clean = df_clean.dropna(subset=['title'])\n",
    "\n",
    "# Create word count features\n",
    "df_clean['abstract_word_count'] = df_clean['abstract'].fillna('').apply(\n",
    "    lambda x: len(str(x).split())\n",
    ")\n",
    "df_clean['title_word_count'] = df_clean['title'].apply(\n",
    "    lambda x: len(str(x).split())\n",
    ")\n",
    "\n",
    "print(f'Cleaned dataset shape: {df_clean.shape}')\n",
    "print(f'Removed {len(df) - len(df_clean)} rows')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Display cleaned data info\n",
    "df_clean.info()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Exploratory Data Analysis"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 4.1 Publication Trends Over Time"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Publications by year\n",
    "year_counts = df_clean['year'].value_counts().sort_index()\n",
    "\n",
    "plt.figure(figsize=(14, 6))\n",
    "plt.bar(year_counts.index, year_counts.values, color='steelblue', edgecolor='black')\n",
    "plt.title('COVID-19 Research Publications by Year', fontsize=16, fontweight='bold')\n",
    "plt.xlabel('Year', fontsize=12)\n",
    "plt.ylabel('Number of Publications', fontsize=12)\n",
    "plt.xticks(rotation=45)\n",
    "plt.grid(axis='y', alpha=0.3)\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "print('Publications by year:')\n",
    "print(year_counts)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 4.2 Top Journals"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Top 15 journals\n",
    "top_journals = df_clean['journal'].value_counts().head(15)\n",
    "\n",
    "plt.figure(figsize=(12, 8))\n",
    "plt.barh(range(len(top_journals)), top_journals.values, color='coral')\n",
    "plt.yticks(range(len(top_journals)), top_journals.index)\n",
    "plt.title('Top 15 Journals Publishing COVID-19 Research', fontsize=16, fontweight='bold')\n",
    "plt.xlabel('Number of Publications', fontsize=12)\n",
    "plt.ylabel('Journal', fontsize=12)\n",
    "plt.gca().invert_yaxis()\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "print('\\nTop 15 journals:')\n",
    "print(top_journals)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 4.3 Source Distribution"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Top sources\n",
    "top_sources = df_clean['source_x'].value_counts().head(10)\n",
    "\n",
    "plt.figure(figsize=(10, 8))\n",
    "plt.pie(top_sources.values, labels=top_sources.index, autopct='%1.1f%%',\n",
    "        startangle=90, colors=sns.color_palette('Set3'))\n",
    "plt.title('Distribution of Papers by Source (Top 10)', fontsize=16, fontweight='bold')\n",
    "plt.axis('equal')\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "print('\\nSource distribution:')\n",
    "print(top_sources)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 4.4 Abstract Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Abstract word count distribution\n",
    "plt.figure(figsize=(12, 6))\n",
    "plt.hist(df_clean['abstract_word_count'][df_clean['abstract_word_count'] > 0], \n",
    "         bins=50, color='skyblue', edgecolor='black')\n",
    "plt.title('Distribution of Abstract Word Counts', fontsize=16, fontweight='bold')\n",
    "plt.xlabel('Word Count', fontsize=12)\n",
    "plt.ylabel('Frequency', fontsize=12)\n",
    "plt.axvline(df_clean['abstract_word_count'].mean(), color='red', \n",
    "            linestyle='--', label=f'Mean: {df_clean[\"abstract_word_count\"].mean():.0f}')\n",
    "plt.legend()\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "print(f'Average abstract length: {df_clean[\"abstract_word_count\"].mean():.1f} words')\n",
    "print(f'Median abstract length: {df_clean[\"abstract_word_count\"].median():.1f} words')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 4.5 Text Analysis - Word Cloud"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Generate word cloud from titles\n",
    "all_titles = ' '.join(df_clean['title'].dropna().astype(str).tolist())\n",
    "\n",
    "wordcloud = WordCloud(\n",
    "    width=1600,\n",
    "    height=800,\n",
    "    background_color='white',\n",
    "    colormap='viridis',\n",
    "    max_words=100\n",
    ").generate(all_titles)\n",
    "\n",
    "plt.figure(figsize=(16, 8))\n",
    "plt.imshow(wordcloud, interpolation='bilinear')\n",
    "plt.axis('off')\n",
    "plt.title('Most Common Words in Paper Titles', fontsize=18, fontweight='bold', pad=20)\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 4.6 Most Common Words"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Extract common words from titles\n",
    "def get_common_words(text_series, top_n=20):\n",
    "    all_text = ' '.join(text_series.dropna().astype(str).tolist())\n",
    "    words = re.findall(r'\\b[a-z]{4,}\\b', all_text.lower())\n",
    "    \n",
    "    stop_words = {'with', 'from', 'that', 'this', 'have', 'been', 'were',\n",
    "                 'their', 'which', 'about', 'there', 'these', 'would'}\n",
    "    words = [w for w in words if w not in stop_words]\n",
    "    \n",
    "    word_counts = Counter(words)\n",
    "    return word_counts.most_common(top_n)\n",
    "\n",
    "common_words = get_common_words(df_clean['title'], top_n=20)\n",
    "words_df = pd.DataFrame(common_words, columns=['Word', 'Count'])\n",
    "\n",
    "plt.figure(figsize=(12, 8))\n",
    "plt.barh(range(len(words_df)), words_df['Count'], color='teal')\n",
    "plt.yticks(range(len(words_df)), words_df['Word'])\n",
    "plt.title('Top 20 Most Common Words in Titles', fontsize=16, fontweight='bold')\n",
    "plt.xlabel('Frequency', fontsize=12)\n",
    "plt.ylabel('Word', fontsize=12)\n",
    "plt.gca().invert_yaxis()\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "print('\\nTop 20 words:')\n",
    "print(words_df)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Summary Statistics"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print('='*60)\n",
    "print('CORD-19 DATASET SUMMARY')\n",
    "print('='*60)\n",
    "print(f'\\nTotal papers analyzed: {len(df_clean):,}')\n",
    "print(f'Date range: {int(df_clean[\"year\"].min())} - {int(df_clean[\"year\"].max())}')\n",
    "print(f'Unique journals: {df_clean[\"journal\"].nunique():,}')\n",
    "print(f'Papers with abstracts: {df_clean[\"abstract\"].notna().sum():,}')\n",
    "print(f'Average abstract length: {df_clean[\"abstract_word_count\"].mean():.1f} words')\n",
    "print(f'\\nMost productive year: {int(df_clean[\"year\"].mode()[0])}')\n",
    "print(f'Top journal: {df_clean[\"journal\"].mode()[0]}')\n",
    "print(f'Top source: {df_clean[\"source_x\"].mode()[0]}')\n",
    "print('='*60)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Export Cleaned Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Save cleaned dataset\n",
    "df_clean.to_csv('metadata_cleaned.csv', index=False)\n",
    "print('Cleaned dataset saved as metadata_cleaned.csv')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Conclusions\n",
    "\n",
    "### Key Findings:\n",
    "1. **Publication Trends**: There was a significant surge in COVID-19 research publications during 2020-2021\n",
    "2. **Research Sources**: Multiple databases contribute to the dataset, showing collaborative research efforts\n",
    "3. **Journal Distribution**: Research is published across diverse journals, indicating broad scientific interest\n",
    "4. **Common Themes**: Based on title analysis, key research areas include viral transmission, treatment, and public health\n",
    "\n",
    "### Next Steps:\n",
    "- Build interactive Streamlit dashboard\n",
    "- Implement advanced text analysis\n",
    "- Add temporal trend analysis\n",
    "- Create author network visualization"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}